<a href="https://colab.research.google.com/github/atulcoin/Simple-Face-Recognition-/blob/master/final_face_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pafy
!pip install imutils pafy youtube-dl

     |████████████████████████████████| 1.8MB 2.8MB/s 


In [2]:
import imutils
import cv2
import pafy
import youtube_dl
from google.colab.patches import cv2_imshow

In [3]:
url = "https://www.youtube.com/watch?v=hoNb6HuNmU0"
vPafy = pafy.new(url)

play = vPafy.getbest(preftype="mp4")

In [ ]:
cap = cv2.VideoCapture(play.url)
print(cap.isOpened()) # if in cap we have entered wrong file path then it will give false else true
while(cap.isOpened()):    # TO CAPTURE FRAMES OF CAMERA TO VIEW AS VIDEO 
  #  print(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) # TO PRINT HEIGHT or property OF THE FRAME 
  #  print(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) # TO PRINT WIDTHE OF THE FRAME 
    face_dec = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

    ret, frame= cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # here i am printing gray image as output
    faces = face_dec.detectMultiScale(gray, scaleFactor =1.05, minNeighbors=5)

    for x,y,w,h in faces:
        gray = cv2.rectangle(gray, (x,y), (x+w,y+h), (0,255,0),2)
        cv2_imshow(gray)
    if cv2.waitKey(1) == 13:  #13 is the asci code of enter key
        break
cap.release()
cv2.destroyAllWindows()

True


error: ignored

training of model with custom data set
**bold text**

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
from PIL import Image
import os
import numpy as np

bpath = '/content/drive/My Drive/part1face2'
image_dir = os.path.join(bpath, "train")

In [ ]:
data_label = []
data_x = []

#count = 0

current_id=0
label_id={}


for root, dirs, files in os.walk(image_dir):
  for file in files:
    if file.endswith("png") or file.endswith("jpg"):
      label = os.path.basename(root).replace(" ","-").lower() #we can replace os.path.dirname(path) with root also
      path = os.path.join(root,file)
            #print(label,path)
            
      if not label in label_id:
        label_id[label] = current_id
        current_id +=1
      id_= label_id[label]
      print(id_)
        #img = cv2.imread(path, 1)
      pil_image=  Image.open(path).convert("L") # L to convert image in grey pil image is used to tell sysetem that the file in location is image file
      image_arra=  np.array(pil_image,"uint8")
            #print(image_arra)
            
      data_label.append(id_)
      data_x.append(image_arra)
data_label = np.asarray(data_label, dtype=np.int32)
data_x = np.asarray(data_x, dtype=np.int32)
#model = cv2.face.LBPHFaceRecognizer_create()

#model.train(np.asarray(x_train), np.asarray(y_label))
#with open("labels.pickle", 'wb') as f:
   # pickle.dump(label_id, f)
print("train complete")
print(data_x)
#model.save('save_face.xml')


In [7]:
data_x.shape , data_label.shape

((600, 200, 200), (600,))

In [8]:
data_x = data_x.astype('float32')
#y_label =y_label.astype('float32')

In [9]:
data_x /= 255
#y_label /=255

i have split data sets into train and test

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_x, data_label, train_size=0.80, random_state=365)

In [11]:
X_train.shape, X_test.shape, 

((480, 200, 200), (120, 200, 200))

creating validation data set

In [12]:
X_train, X_vali, y_train, y_vali = train_test_split(X_train, y_train, train_size=0.80, random_state=365)

In [13]:
X_train.shape,X_test.shape,X_vali.shape

((384, 200, 200), (120, 200, 200), (96, 200, 200))

In [14]:
y_train[300]

4

In [15]:
X_train = np.expand_dims(X_train, -1)
X_test = np.expand_dims(X_test, -1)
X_vali = np.expand_dims(X_vali, -1)

In [16]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import keras
import tensorflow as tf


In [ ]:
input_size = 400
output_size = 6
hidden_layer_size = 50
model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(200, 200, 1)),
        tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
        tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
       tf.keras.layers.Dense(output_size, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_vali, y_vali), verbose =2)
print ("training complete")

Epoch 1/10
12/12 - 0s - loss: 2.0555 - accuracy: 0.5286 - val_loss: 0.4808 - val_accuracy: 0.9062
Epoch 2/10
12/12 - 0s - loss: 0.3592 - accuracy: 0.9062 - val_loss: 0.1327 - val_accuracy: 0.9896
Epoch 3/10
12/12 - 0s - loss: 0.1100 - accuracy: 0.9870 - val_loss: 0.0461 - val_accuracy: 1.0000
Epoch 4/10
12/12 - 0s - loss: 0.0564 - accuracy: 0.9922 - val_loss: 0.0284 - val_accuracy: 1.0000
Epoch 5/10
12/12 - 0s - loss: 0.0364 - accuracy: 0.9974 - val_loss: 0.0157 - val_accuracy: 1.0000
Epoch 6/10
12/12 - 0s - loss: 0.0304 - accuracy: 0.9974 - val_loss: 0.0134 - val_accuracy: 1.0000
Epoch 7/10
12/12 - 0s - loss: 0.0248 - accuracy: 0.9974 - val_loss: 0.0109 - val_accuracy: 1.0000
Epoch 8/10
12/12 - 0s - loss: 0.0205 - accuracy: 0.9974 - val_loss: 0.0092 - val_accuracy: 1.0000
Epoch 9/10
12/12 - 0s - loss: 0.0191 - accuracy: 0.9974 - val_loss: 0.0083 - val_accuracy: 1.0000
Epoch 10/10
12/12 - 0s - loss: 0.0169 - accuracy: 0.9974 - val_loss: 0.0071 - val_accuracy: 1.0000
training complete


In [37]:
X_test.shape

(120, 200, 200, 1)

In [17]:
model = keras.models.Sequential([
                         keras.layers.Conv2D(filters=32, kernel_size=3, strides=(1,1), padding='valid', activation='relu', input_shape=[200,200,1]),
                         keras.layers.MaxPooling2D(pool_size=(2, 2)),
                         keras.layers.Flatten(),
                         keras.layers.Dense(units=128, activation='relu'),
                         keras.layers.Dense(units=6, activation='softmax'),
])

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 198, 198, 32)      320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 99, 99, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 313632)            0         
_________________________________________________________________
dense (Dense)                (None, 128)               40145024  
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 774       
Total params: 40,146,118
Trainable params: 40,146,118
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [20]:
model.fit(X_train, y_train, epochs=9, batch_size=100, verbose=1, validation_data=(X_vali, y_vali))

Epoch 1/9
4/4 [==============================] - 10s 2s/step - loss: 9.2205 - accuracy: 0.2578 - val_loss: 15.7396 - val_accuracy: 0.3646
Epoch 2/9
4/4 [==============================] - 7s 2s/step - loss: 10.3246 - accuracy: 0.3464 - val_loss: 7.1616 - val_accuracy: 0.5104
Epoch 3/9
4/4 [==============================] - 6s 2s/step - loss: 4.2781 - accuracy: 0.7188 - val_loss: 3.6501 - val_accuracy: 0.5625
Epoch 4/9
4/4 [==============================] - 6s 2s/step - loss: 1.5734 - accuracy: 0.7370 - val_loss: 0.3601 - val_accuracy: 0.8021
Epoch 5/9
4/4 [==============================] - 6s 2s/step - loss: 0.3691 - accuracy: 0.8333 - val_loss: 0.4078 - val_accuracy: 0.7917
Epoch 6/9
4/4 [==============================] - 6s 2s/step - loss: 0.2321 - accuracy: 0.9219 - val_loss: 0.1340 - val_accuracy: 1.0000
Epoch 7/9
4/4 [==============================] - 6s 2s/step - loss: 0.1131 - accuracy: 0.9870 - val_loss: 0.0990 - val_accuracy: 0.9896
Epoch 8/9
4/4 [==============================

training accuracy is 100% may be of data set which i have used

In [22]:
model.predict(np.expand_dims(X_test[0], axis=0)).round(2)

array([[0.99, 0.  , 0.  , 0.  , 0.  , 0.  ]], dtype=float32)

In [25]:
np.argmax(model.predict(np.expand_dims(X_test[55], axis=0)).round(2))

2

In [26]:
y_test[55]

2

In [27]:
model.evaluate(X_test,y_test)

4/4 [==============================] - 1s 167ms/step - loss: 0.0283 - accuracy: 1.0000


[0.028307469561696053, 1.0]